In [98]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime
# import hvplot.pandas
# import matplotlib.pyplot as plt
# from sklearn import svm
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from pandas.tseries.offsets import DateOffset
# from sklearn.metrics import classification_report

from utils.helpers import alpaca_symbol_data

# For creating the DB
database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

# prevent an ssl failure
ssl._create_default_https_context = ssl._create_unverified_context

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# There are 2 tables on the Wikipedia page
# we want the first table
first_table = payload[0]
second_table = payload[1]

df500table = first_table

sp500Tickers = df500table['Symbol'].values.tolist()
df500sectors = df500table.drop(columns={'Security','SEC filings','GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK','Founded'})
df500sectors.set_index('Symbol', inplace=True)
# print(sp500Tickers)

In [99]:
count = 0
tickers_0 = []
tickers_1 = []
tickers_2 = []
tickers_3 = []
tickers_4 = []
tickers_5 = []


size = len(sp500Tickers)
print(size)

for each_ticker in sp500Tickers:
    if count <= 99:
        tickers_0.append(each_ticker)
    if count >= 100 and count <= 199:
        tickers_1.append(each_ticker)
    if count >= 200 and count <= 299:
        tickers_2.append(each_ticker)
    if count >= 300 and count <= 399:
        tickers_3.append(each_ticker)
    if count >= 400 and count <= 499:
        tickers_4.append(each_ticker)
    if count >= 500:
        tickers_5.append(each_ticker)
    count = count + 1
# print(count)


print(len(tickers_0))
# print(tickers_0)
print(len(tickers_1))
# print(tickers_1)
print(len(tickers_2))
# print(tickers_2)
print(len(tickers_3))
# print(tickers_3)
print(len(tickers_4))
# print(tickers_4)
print(len(tickers_5))
# print(tickers_5)


504
100
100
100
100
100
4


In [100]:
# Establish a Baseline Performance
# Import the OHLCV dataset into a Pandas DataFrame
tickerList =[tickers_0, tickers_1, tickers_2, tickers_3, tickers_4, tickers_5]
# Set the ticker
tickers = tickers_0

# Set start datetimes n years from today
years_back = 1

# Get the symbol_data and store it into a variable ohlcv_df
# ohlcv_df = alpaca_symbol_data(tickers, years_back)
ohlcv_df = pd.DataFrame()

StockCount = 100
for tickers in tickerList:
    ohlcv_df = alpaca_symbol_data(tickers, years_back)
    display(ohlcv_df.head())
    ohlcv_df.to_sql('stocks_'+ str(StockCount), engine)
    StockCount +=100


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-12 04:00:00+00:00,23.25,23.3200,22.720,22.92,26250945,120661,22.891158,AAL
2021-04-13 04:00:00+00:00,22.20,22.6500,21.660,22.56,42928205,201169,22.159439,AAL
2021-04-14 04:00:00+00:00,22.95,23.3500,22.200,22.37,33828696,150663,22.666991,AAL
2021-04-15 04:00:00+00:00,22.63,22.6669,21.695,22.14,32467782,154150,22.041880,AAL
2021-04-16 04:00:00+00:00,22.09,22.3600,21.870,22.02,24850633,120722,22.033054,AAL


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-12 04:00:00+00:00,36.37,37.57,36.280,37.53,5383535,42847,37.254376,CAG
2021-04-13 04:00:00+00:00,37.53,37.64,37.180,37.22,3251258,24914,37.329583,CAG
2021-04-14 04:00:00+00:00,37.21,37.42,36.930,37.25,3006938,26273,37.236985,CAG
2021-04-15 04:00:00+00:00,37.28,37.62,37.280,37.58,1880621,18730,37.519448,CAG
2021-04-16 04:00:00+00:00,37.65,37.74,37.145,37.55,2782092,20150,37.499102,CAG


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-12 04:00:00+00:00,132.34,132.76,131.43,132.35,548262,11021,132.118394,AJG
2021-04-13 04:00:00+00:00,132.31,134.69,131.47,133.95,540140,8933,133.665679,AJG
2021-04-14 04:00:00+00:00,133.76,134.63,132.48,132.98,567852,9479,133.181328,AJG
2021-04-15 04:00:00+00:00,133.44,135.14,133.44,135.06,639762,8903,134.717899,AJG
2021-04-16 04:00:00+00:00,135.86,137.44,134.80,136.80,754536,10610,136.625447,AJG


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-12 04:00:00+00:00,127.32,128.86,127.05,128.86,1069622,17786,128.449548,DGX
2021-04-13 04:00:00+00:00,129.06,130.77,128.12,128.39,982240,11817,128.927925,DGX
2021-04-14 04:00:00+00:00,128.36,128.76,126.65,127.34,1272952,14743,127.409740,DGX
2021-04-15 04:00:00+00:00,127.80,129.79,127.80,129.15,1009083,15007,128.992768,DGX
2021-04-16 04:00:00+00:00,130.14,130.57,128.80,130.11,1437692,13861,130.001567,DGX


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-12 04:00:00+00:00,229.52,229.85,227.1500,228.76,5436756,89726,228.500778,CRM
2021-04-13 04:00:00+00:00,229.00,233.49,228.5000,232.00,4606178,83914,231.865410,CRM
2021-04-14 04:00:00+00:00,232.85,233.14,227.6600,228.15,4074835,73814,229.694322,CRM
2021-04-15 04:00:00+00:00,229.87,233.08,229.5000,233.00,4636395,74705,231.551743,CRM
2021-04-16 04:00:00+00:00,234.00,234.00,229.8935,231.91,4524141,66251,232.006370,CRM


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-12 04:00:00+00:00,168.69,169.810,167.69,168.33,1289382,12627,168.490242,ZBH
2021-04-13 04:00:00+00:00,167.49,169.140,167.18,168.47,876859,11954,168.092535,ZBH
2021-04-14 04:00:00+00:00,168.91,170.525,168.06,168.31,892191,16969,169.249714,ZBH
2021-04-15 04:00:00+00:00,168.60,172.340,168.60,171.76,933507,12861,171.605692,ZBH
2021-04-16 04:00:00+00:00,172.74,173.220,170.38,171.88,711084,10832,171.632120,ZBH


In [101]:
filepath = Path('./resources/ohlcv_df_df.csv') 
filepath.parent.mkdir(parents=True, exist_ok=True) 
ohlcv_df.to_csv(filepath)

PermissionError: [Errno 13] Permission denied: 'resources\\ohlcv_df_df.csv'

In [194]:
# Save to DB, be careful here
# ohlcv_df.to_sql('stocks_600', engine)

In [84]:
# Read from DB
Stocks100 = pd.read_sql_table('stocks_100', engine, parse_dates='timestamp')
Stocks200 = pd.read_sql_table('stocks_200', engine, parse_dates='timestamp')
Stocks300 = pd.read_sql_table('stocks_300', engine, parse_dates='timestamp')
Stocks400 = pd.read_sql_table('stocks_400', engine, parse_dates='timestamp')
Stocks500 = pd.read_sql_table('stocks_500', engine, parse_dates='timestamp')
Stocks600 = pd.read_sql_table('stocks_600', engine, parse_dates='timestamp')

In [85]:
Stocks = pd.concat([Stocks100, Stocks200, Stocks300, Stocks400, Stocks500, Stocks600], axis=0)
Stocks['daily return'] = (Stocks['close'] - Stocks['open'])/Stocks['open']
StocksTall = Stocks.drop(columns={'open','high','low','close','volume','trade_count','vwap'})

In [86]:
StocksTall



,timestamp,symbol,daily return
0,2021-04-12 04:00:00,AAL,-0.014194
1,2021-04-13 04:00:00,AAL,0.016216
2,2021-04-14 04:00:00,AAL,-0.025272
3,2021-04-15 04:00:00,AAL,-0.021653
4,2021-04-16 04:00:00,AAL,-0.003169
...,...,...,...
1011,2022-04-05 04:00:00,ZTS,-0.001801
1012,2022-04-06 04:00:00,ZTS,0.017113
1013,2022-04-07 04:00:00,ZTS,0.035199
1014,2022-04-08 04:00:00,ZTS,0.011628


In [87]:
StocksWide = pd.pivot_table(StocksTall, columns='symbol', index = 'timestamp')

In [88]:
StocksWide.head()

daily return                                          \
symbol                         A       AAL       AAP      AAPL      ABBV   
timestamp                                                                  
2021-04-12 04:00:00    -0.001293 -0.014194  0.021788 -0.007705  0.003988   
2021-04-13 04:00:00     0.010410  0.016216 -0.007205  0.015125  0.001759   
2021-04-14 04:00:00    -0.008490 -0.025272  0.001600 -0.022000 -0.003013   
2021-04-15 04:00:00     0.004598 -0.021653  0.006214  0.004781  0.008491   
2021-04-16 04:00:00     0.003954 -0.003169  0.008574 -0.003308  0.003907   

                                                                       ...  \
symbol                    ABC      ABMD       ABT       ACN      ADBE  ...   
timestamp                                                              ...   
2021-04-12 04:00:00  0.008408 -0.005014  0.003981  0.005490  0.013559  ...   
2021-04-13 04:00:00  0.001968 -0.011522  0.016192 -0.004815  0.015665  ...   
2021-04-14 04:00:00  0.009064  0.002824 -0.006947  0.000844 -0.006556  ...   
2021-04-15 04:00:00  0.005066  0.012594  0.013658 -0.003878  0.014951  ...   
2021-04-16 04:00:00  0.001590 -0.026277 -0.000322  0.006312  0.002310  ...   

                                                                       \
symbol                   WYNN       XEL       XOM      XRAY       XYL   
timestamp                                                               
2021-04-12 04:00:00 -0.007863  0.009064 -0.022026 -0.002178  0.003796   
2021-04-13 04:00:00  0.011212  0.013618  0.008143  0.002355 -0.010348   
2021-04-14 04:00:00 -0.007537  0.004826  0.017936  0.018744  0.007477   
2021-04-15 04:00:00 -0.014030  0.020566 -0.004716 -0.005371  0.001019   
2021-04-16 04:00:00 -0.005785  0.001423 -0.010651 -0.000306 -0.005585   

                                                                       
symbol                    YUM       ZBH      ZBRA      ZION       ZTS  
timestamp                                                              
2021-04-12 04:00:00  0.012348 -0.002134 -0.003978  0.012701  0.020407  
2021-04-13 04:00:00  0.008414  0.005851 -0.010332 -0.007684  0.007830  
2021-04-14 04:00:00  0.005360 -0.003552 -0.005917  0.015529 -0.005670  
2021-04-15 04:00:00 -0.007259  0.018743  0.005353 -0.011160  0.009272  
2021-04-16 04:00:00 -0.004889 -0.004979  0.004952 -0.004079  0.008179  

[5 rows x 504 columns]

In [89]:
dailyreturns = [ ]
for column in StocksWide.columns.values:
    dailyreturns.append ( StocksWide [ column ].tolist () )

In [90]:
tickers = []
count = 0
for column in StocksWide.columns:
    tickers.append (StocksWide.columns[count][1])
    count += 1

In [91]:
dateTime = StocksWide.index.to_list()
StocksWideFixed = pd.DataFrame()
StocksWideFixed['dateTime'] = dateTime

In [92]:
StockCount = 0
while StockCount < 504:
    StocksWideFixed[tickers[StockCount]] = dailyreturns[StockCount]
    StockCount += 1
StocksWideFixed = StocksWideFixed.copy()

C:\Users\canav\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until


In [93]:
StocksWideFixed.head()

,dateTime,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2021-04-12 04:00:00,-0.001293,-0.014194,0.021788,-0.007705,0.003988,0.008408,-0.005014,0.003981,0.005490,...,-0.007863,0.009064,-0.022026,-0.002178,0.003796,0.012348,-0.002134,-0.003978,0.012701,0.020407
1,2021-04-13 04:00:00,0.010410,0.016216,-0.007205,0.015125,0.001759,0.001968,-0.011522,0.016192,-0.004815,...,0.011212,0.013618,0.008143,0.002355,-0.010348,0.008414,0.005851,-0.010332,-0.007684,0.007830
2,2021-04-14 04:00:00,-0.008490,-0.025272,0.001600,-0.022000,-0.003013,0.009064,0.002824,-0.006947,0.000844,...,-0.007537,0.004826,0.017936,0.018744,0.007477,0.005360,-0.003552,-0.005917,0.015529,-0.005670
3,2021-04-15 04:00:00,0.004598,-0.021653,0.006214,0.004781,0.008491,0.005066,0.012594,0.013658,-0.003878,...,-0.014030,0.020566,-0.004716,-0.005371,0.001019,-0.007259,0.018743,0.005353,-0.011160,0.009272
4,2021-04-16 04:00:00,0.003954,-0.003169,0.008574,-0.003308,0.003907,0.001590,-0.026277,-0.000322,0.006312,...,-0.005785,0.001423,-0.010651,-0.000306,-0.005585,-0.004889,-0.004979,0.004952,-0.004079,0.008179


In [94]:
StocksWideFixed.to_sql('StocksWideFixed', engine)